In [ ]:
import openai
from tqdm import tqdm
import os
import pandas as pd
import snowflake.connector

api_key_file = 'apikey.txt'
if os.path.isfile(api_key_file):
    with open(api_key_file) as f:
        openai.api_key = f.readline()
else:
    print(f"Error: {api_key_file} not found.")

OPENAI_API_KEY = openai.api_key

#login to snowflake db
con = snowflake.connector.connect(user='vishal.kumar@scale.com',
                                 account='pxa65918',
                                 authenticator='externalbrowser',
                                 warehouse='COMPUTE_WH',
                                 database='SCALE_CRAWLER',
                                 role='GENERAL_RO')

cs = con.cursor()

In [ ]:
#Get data from snowflake
sql = f'''
Select 
task,
t.params:templateVariables.image_type::string as Image_Type,
t.params:templateVariables.pr_type::string as pr_Type,
t.params:templateVariables.image_url::string as Image_Url,
ta.response:responses[2].context.response.annotations.low_quality.response[0][0]::string as Image_Usable,
ta.response:responses[3].context.response.annotations.prompt.response[0]::string as Prompt,
ta.response:responses[3].context.response.annotations.response.response[0]::string as Response
from scale_prod.public.taskattempts ta join scale_prod.public.tasks t on ta.task=t._id
where ta.project='65e23dd219c580a16e44e374'
and prompt is not null
'''
cs.execute(sql)
idf = cs.fetch_pandas_all()

In [ ]:
def vision_gpt(image_url, prompt):
    import openai
    client = openai
    response = client.completions.create(
        model="gpt-4-vision-preview",
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": image_url,
                            "detail": "high"
                        },
                    },
                ],
            }
        ],
        max_tokens=300,
    )
    return response

In [ ]:
#add a another blank column to idf called SOTA_RESPONSE
idf['SOTA_RESPONSE'] = None
#create a for loop to iterate over the rows of idf
for i, row in tqdm(idf.iterrows(), total=idf.shape[0]):
    #save current row idf column IMAGE_URL to a variable called image_url
    image_url = row['IMAGE_URL']
    #save current row idf column PROMPT to a variable called prompt
    prompt = row['PROMPT']
    #call the vision_gpt function with image_url and prompt as arguments
    response = vision_gpt(image_url, prompt)
    response = response.choices[0]
    idf.at[i, 'SOTA_RESPONSE'] = response
    print(response)

#save idf to a csv file
idf.to_csv('Nightingale_MM_SFT_DI_SOTA.csv', index=False)